# Advanced Job Submission and Management

Launch this tutorial in a Jupyter Notebook on Binder: 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/htcondor/htcondor-python-bindings-tutorials/master?urlpath=lab/tree/Advanced-Job-Submission-And-Management.ipynb)

The two most common HTCondor command line tools are `condor_q` and `condor_submit`.
In the previous module, we learned about the `xquery()` method that corresponds to `condor_q`. Here, we will learn the Python binding equivalent of `condor_submit` in greater detail.

We start by importing the relevant modules:

In [1]:
import htcondor

Submitting Jobs
---------------

We will submit jobs utilizing the dedicated `Submit` object.

`Submit` objects consist of key-value pairs. Unlike ClassAds, the values do not have an inherent type (such as strings, integers, or booleans); they are evaluated with macro expansion at submit time. Where reasonable, they behave like Python dictionaries:

In [2]:
sub = htcondor.Submit({"foo": "1", "bar": "2", "baz": "$(foo)"})
print(sub)

foo = 1
bar = 2
baz = $(foo)



In [3]:
sub["qux"] = 3
print(sub)

foo = 1
bar = 2
baz = $(foo)
qux = 3



In [4]:
print(sub.expand("baz"))

1


The available attributes and their semantics are documented in the `condor_submit` [manual](https://htcondor.readthedocs.io/en/latest/man-pages/condor_submit.html),
sowe won’t repeat them here.
A minimal realistic submit object may look like the following:

In [5]:
sub = htcondor.Submit({
    "executable": "/bin/sleep",
    "arguments": "5m"
})

To go from a submit object to job in a schedd, one must use the `submit` method of a `htcondor.Schedd`:

In [6]:
schedd = htcondor.Schedd()          # create a schedd object connected to the local schedd
submit_result = schedd.submit(sub)  # queue one job
print(submit_result.cluster())      # print the job's ClusterId

1


By default, each invocation of `submit` will submit a single job.  A more common use case is to submit many jobs at once - often identical.  Suppose we don't want to submit a single "sleep" job, but 10; instead of writing a `for`-loop around the `submit` method, we can use the `count` argument:

In [7]:
submit_result = schedd.submit(sub, count=10)

print(submit_result.cluster())

2


We can now query for those jobs in the queue:

In [8]:
schedd.query(
    constraint='ClusterId =?= {}'.format(submit_result.cluster()),
    projection=["ClusterId", "ProcId", "JobStatus", "EnteredCurrentStatus"],
)

[[ ClusterId = 2; ProcId = 0; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 1; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 2; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 3; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 4; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 5; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 6; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 7; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 8; EnteredCurrentStatus = 1631798050; JobStatus = 1; ServerTime = 1631798050 ],
 [ ClusterId = 2; ProcId = 9; Entered

It's not entirely useful to submit many identical jobs -- but rather each one needs to vary slightly based on its ID (the "process ID") within the job cluster.  For this, the `Submit` object in Python behaves  similarly to submit files: references within the submit command are evaluated as macros at submit time.

For example, suppose we want the argument to `sleep` to vary based on the process ID:

In [9]:
sub = htcondor.Submit({"executable": "/bin/sleep", "arguments": "$(Process)s"})

Here, the `$(Process)` string will be substituted with the process ID at submit time.

In [10]:
submit_result = schedd.submit(sub, count=10)

print(submit_result.cluster())

schedd.query(
    constraint='ClusterId=?={}'.format(submit_result.cluster()),
    projection=["ClusterId", "ProcId", "JobStatus", "Args"],
)

3


[[ Args = "0s"; ClusterId = 3; ProcId = 0; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "1s"; ClusterId = 3; ProcId = 1; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "2s"; ClusterId = 3; ProcId = 2; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "3s"; ClusterId = 3; ProcId = 3; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "4s"; ClusterId = 3; ProcId = 4; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "5s"; ClusterId = 3; ProcId = 5; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "6s"; ClusterId = 3; ProcId = 6; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "7s"; ClusterId = 3; ProcId = 7; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "8s"; ClusterId = 3; ProcId = 8; JobStatus = 1; ServerTime = 1631798050 ],
 [ Args = "9s"; ClusterId = 3; ProcId = 9; JobStatus = 1; ServerTime = 1631798050 ]]

The macro evaluation behavior (and the various usable tricks and techniques) are identical between the python bindings and the `condor_submit` executable.

## Managing Jobs

Once a job is in queue, the schedd will try its best to execute it to completion. There are several cases where a user may want to interrupt the normal flow of jobs. Perhaps the results are no longer needed; perhaps the job needs to be edited to correct a submission error. These actions fall under the purview of _job management_.

There are two `Schedd` methods dedicated to job management:

* `edit()`: Change an attribute for a set of jobs to a given expression. If invoked within a transaction, multiple calls to `edit` are visible atomically.
   * The set of jobs to change can be given as a ClassAd expression. If no jobs match the filter, _then an exception is thrown_.
* `act()`: Change the state of a job to a given state (remove, hold, suspend, etc).

Both methods take a _job specification_: either a ClassAd expression (such as `Owner =?= "janedoe"`)
or a list of job IDs (such as `["1.1", "2.2", "2.3"]`).
The `act` method takes an argument from the `JobAction` enum.
The commonly-used values are:

* `Hold`: put a job on hold, vacating a running job if necessary.  A job will stay in the hold
   state until explicitly acted upon by the admin or owner.
* `Release`: Release a job from the hold state, returning it to Idle.
* `Remove`: Remove a job from the Schedd's queue, cleaning it up first on the remote host (if running).
   This requires the remote host to acknowledge it has successfully vacated the job, meaning ``Remove`` may
   not be instantaneous.
* `Vacate`: Cause a running job to be killed on the remote resource and return to idle state.  With
  `Vacate`, jobs may be given significant time to cleanly shut down.

Here's an example of job management in action:

In [11]:
submit_result = schedd.submit(sub, count=5)  # queues 5 copies of this job
schedd.edit([f"{submit_result.cluster()}.{idx}" for idx in range(2)], "foo", '"bar"')  # sets attribute foo to the string "bar" for the first two jobs
    
for ad in schedd.xquery(
    constraint=f"ClusterId == {submit_result.cluster()}",
    projection=["ProcId", "JobStatus", "foo"],
):
    print(repr(ad))

[ ServerTime = 1631798050; ProcId = 0; JobStatus = 1; foo = "bar" ]
[ ServerTime = 1631798050; ProcId = 1; JobStatus = 1; foo = "bar" ]
[ ServerTime = 1631798050; ProcId = 2; JobStatus = 1 ]
[ ServerTime = 1631798050; ProcId = 3; JobStatus = 1 ]
[ ServerTime = 1631798050; ProcId = 4; JobStatus = 1 ]


In [12]:
schedd.act(htcondor.JobAction.Hold, f"ClusterId == {submit_result.cluster()} && ProcId >= 2")

for ad in schedd.xquery(
    constraint=f"ClusterId == {submit_result.cluster()}",
    projection=["ProcId", "JobStatus", "foo"],
):
    print(repr(ad))

[ ServerTime = 1631798050; ProcId = 0; JobStatus = 1; foo = "bar" ]
[ ServerTime = 1631798050; ProcId = 1; JobStatus = 1; foo = "bar" ]
[ ServerTime = 1631798051; ProcId = 2; JobStatus = 5 ]
[ ServerTime = 1631798051; ProcId = 3; JobStatus = 5 ]
[ ServerTime = 1631798051; ProcId = 4; JobStatus = 5 ]


Finally, let's clean up after ourselves (this will remove all of the jobs you own from the queue).

In [13]:
import getpass

schedd.act(htcondor.JobAction.Remove, f'Owner == "{getpass.getuser()}"')

[ TotalJobAds = 26; TotalPermissionDenied = 0; TotalAlreadyDone = 0; TotalNotFound = 0; TotalSuccess = 26; TotalChangedAds = 1; TotalBadStatus = 0; TotalError = 0 ]

## That's It!

You've made it through the very basics of the Python bindings.  While there are many other features the Python
module has to offer, we have covered enough to replace the command line tools of `condor_q`, `condor_submit`,
`condor_status`, `condor_rm` and others.